In [2]:
from port_dash import *
import pandas as pd
import os
import yfinance as yf
import numpy as np
import requests
import json
from datetime import datetime
from threading import Timer
import ipywidgets as widgets
from IPython.display import display
import hvplot.pandas
import web3, solc, json
from solc import compile_standard
from web3.contract import ConciseContract, ContractCaller
from web3 import Web3
from web3.middleware import geth_poa_middleware
from web3.gas_strategies.time_based import medium_gas_price_strategy
from web3.auto.gethdev import w3

ModuleNotFoundError: No module named 'port_dash'

In [2]:
# 1 of the following networks must be selected
# For local network
#w3 = Web3(Web3.HTTPProvider("http://127.0.0.1:8545"))

# For Kovan live testnet
w3 = Web3(Web3.HTTPProvider("https://kovan.infura.io/v3/c939afba949d4aa2903246e8029e4d49"))


#w3.middleware_onion.inject(geth_poa_middleware, layer=0)
#w3.eth.setGasPriceStrategy(medium_gas_price_strategy)

In [3]:
# Ignore
def calculate_sharpe(ticker, time_frame):
    
    stock = yf.Ticker(ticker)
    df = stock.history(period=time_frame)['Close'].to_frame()
    
    df['Daily Returns'] = df['Close'].diff()
    df.dropna(inplace=True)
    
    risk_free = 0.0067
    avg_return = df['Daily Returns'].mean()
    vol = df['Daily Returns'].std()
    sharpe = (avg_return - risk_free) / vol * np.sqrt(252)
    
    return sharpe

In [4]:
weights = [0.40, 0.25, 0.15, 0.10, 0.05, 0.05]

conservative_stocks = ['NEM', 'DG', 'REGN', 'DXCM', 'AAPL']
moderate_stocks = ['DXCM', 'NVDA', 'ODFL', 'DG', 'REGN', 'AAPL']
aggressive_stocks = ['NVDA', 'DXCM', 'AAPL', 'SWKS', 'ODFL']

In [5]:
# Ignore
def cum_return(stock_list, weight_list, time_frame):
    
    df = pd.DataFrame()
    
    for stock in stock_list:
        ticker = yf.Ticker(stock)
        df[f'{stock} Close'] = ticker.history(period=time_frame)['Close']
        df.dropna(inplace=True)
        
    cumulative_returns = []    
    
    for date in df.index:
        index = 0
        adj_closes = []
        
        for i in df.loc[date]:
            adj_close = i * weights[index]
            adj_closes.append(adj_close)
        
            index += 1
            
        cum_adj_return = sum(adj_closes)
        cumulative_returns.append(cum_adj_return)
        
    df['Portfolio Adj. Close'] = cumulative_returns
    
    df['Daily Adj. Return'] = df['Portfolio Adj. Close'].diff()
    df.dropna(inplace=True)
    
    risk_free = 0.0067
    avg_return = df['Daily Adj. Return'].mean()
    vol = df['Daily Adj. Return'].std()
    sharpe = (avg_return - risk_free) / vol * np.sqrt(252)
        
    return df, sharpe

In [6]:
# Creates contract instances for deployer, token, and token sale
# Need to update address for deployer contract if redeployed
deployer_contract = w3.eth.contract(address=deployer_contract_address, abi=deployer_abi)

coinA_contract = w3.eth.contract(address=deployer_contract.caller.token_address(), abi=coin_abi)

CoinA_sale_contract = w3.eth.contract(address=deployer_contract.caller.token_sale_address(), abi=sale_abi)

coin_name = coinA_contract.caller.name()

In [7]:
# API pulls exchange rate - USD/ETH
url = 'https://rest.coinapi.io/v1/exchangerate/ETH/USD'
headers = {'X-CoinAPI-Key' : 'AE657143-7169-4CE9-9923-73207061F320'}
response = requests.get(url, headers=headers).text
eth_rate = json.loads(response)['rate']

In [8]:
############ Widgets ############

# To select portfolio plots/info
risk_selector = widgets.Dropdown(
    options = ['Conservative', 'Moderate', 'Aggressive'],
    value = 'Conservative',
    description = 'Risk Tolerance',
    style = {'description_width': 'initial'},
    disabled = False
)

time = widgets.IntText(
    value = 1,
    description = 'Time Frame'
)

time_selector = widgets.Dropdown(
    options = ['year(s)', 'month(s)', 'day(s)'],
    value = 'year(s)'
)

button = widgets.Button(
    description = 'Update',
    layout = {'border': '1px solid black'}
)

In [9]:
# for use on local blockchain only #

#account_selector = widgets.Dropdown(
#    options = w3.eth.accounts,
#    value = w3.eth.accounts[1],
#    description = 'Account: '
#)

# Must input account address and private key manually on live testnet
account_selector = widgets.Text(
    value = '',
    description = 'Account: ',
    placeholder = 'Enter account address'
)

key_input = widgets.Password(
    value = '',
    description = 'Private Key: ',
    placeholder = 'Enter private key'
)


# Input token amount for purchase
coin_text = widgets.IntText(
    value = 0,
    description = f'{coin_name}s :',
    style = {'description_width': 'initial'},
    disabled = False
)

# Purchase button
coin_button = widgets.Button(
    description = 'Buy!',
    layout ={'border': '1px solid black'},
    disabled = False
)

In [10]:
account_input = widgets.HBox([account_selector, key_input])
purchase_coins = widgets.HBox([coin_text, coin_button])
purchase_from_acct = widgets.VBox([account_input, purchase_coins])

In [11]:
purchase_from_acct

In [12]:
# Show equivalent amount in USD
dollar_conversion = widgets.Text(
    value = "{:,}".format((coin_text.value / (10 ** coinA_contract.caller.decimals())) * eth_rate),
    description = '$',
    disabled = True
)
#(10 ** coinA_contract.caller.decimals()))

# Confirm purchase
confirm_button = widgets.Button(
    description = 'Confirm',
    layout = {'border': '1px solid black'}
)

In [13]:
# Keeps dollar conversion and confirm button hidden until buy button is clicked
confirm_box = widgets.HBox([dollar_conversion, confirm_button])
confirm_box.layout.visibility = 'hidden'
purchase_box = widgets.VBox([purchase_from_acct, confirm_box])

In [14]:
# Displays the buyer's purchased tokens
cons_coin = widgets.IntText(
    value = 0,
    description = f'Your {coin_name}s: ',
    style = {'description_width': 'initial'},
    disabled = True
)

# Remaining Token Supply
cons_coin_supply = widgets.IntText(
    value = CoinA_sale_contract.caller.cap() - CoinA_sale_contract.caller.weiRaised(),
    description = f'Remaining supply of {coin_name}: ',
    style = {'description_width': 'initial'},
    disabled = True
)

In [15]:
# Withdraw Tokens widgets
withdraw_account = widgets.Text(
    value = '',
    description = 'Account: ',
    placeholder = 'Enter account address: ',
    width = '100%'
)

withdraw_key_input = widgets.Password(
    value = '',
    description = 'Private key: ',
    placeholder = 'Enter private key: ',
    layout = widgets.Layout(align_items='center')
)

withdraw_button = widgets.Button(
    description = 'Withdraw Tokens',
    layout={'border': '1px solid black'}
    #layout = widgets.Layout(align_items='center')
)


token_balance = widgets.IntText(
    value = 0,
    description = coinA_contract.caller.symbol(),
    layout = widgets.Layout(align_items='center'),
    disabled = True
)

token_balance_button = widgets.Button(
    description = f'Get {coin_name} Balance',
    layout={'border': '1px solid black'}
    #layout = widgets.Layout(align_items='center')
)

In [16]:
# Withdraw tokens functionality - remains hidden until contract is finalized

account_box = widgets.VBox([withdraw_account], 
                           layout = widgets.Layout(align_items='center'),
                           width = '100%'
                          )
withdraw_box = widgets.VBox([withdraw_key_input, withdraw_button], 
                            layout = widgets.Layout(align_items='center')
                           )
token_balance_box = widgets.VBox([token_balance, token_balance_button], 
                                 layout = widgets.Layout(align_items='center')
                                )
post_sale_box = widgets.AppLayout(
    header = account_box,
    left_sidebar = withdraw_box,
    center = None,
    right_sidebar = token_balance_box,
    footer = None,
    width = '100%'
    #align_items = 'center'
)

post_sale_box.layout.visibility = 'hidden'

In [17]:
coins_purchased_box = widgets.VBox([cons_coin, cons_coin_supply])
coins_box = widgets.VBox([purchase_box, coins_purchased_box])

In [18]:
def generate_portfolio(risk, time_input, time_unit):
    
    if time_unit == 'year(s)':
        time_frame = f'{time_input}y'
        
    elif time_unit == 'month(s)':
        time_frame = f'{time_input}mo'
        
    elif time_unit == 'day(s)':
        time_frame = f'{time_input}d'
    
    if risk == 'Conservative':
        portfolio, sharpe = cum_return(conservative_stocks, weights, time_frame)
        
    elif risk == 'Moderate':
        portfolio, sharpe = cum_return(moderate_stocks, weights, time_frame)
        
    elif risk == 'Aggressive':
        portfolio, sharpe = cum_return(aggressive_stocks, weights, time_frame)
        
    return portfolio

In [19]:
## Output widgets ##

out = widgets.Output()
with out:
    port_df = generate_portfolio(risk_selector.value, time.value, time_selector.value)
    display(port_df['Portfolio Adj. Close'].hvplot())
    

coin_out = widgets.Output()
with coin_out:
    if CoinA_sale_contract.caller.isOpen():
        display(coins_box)
    else:
        pass
    
    
confirm_out = widgets.Output()
with confirm_out:
    if CoinA_sale_contract.caller.hasClosed():
        pass
    else:
        display(confirm_box)
        
        
withdraw_out = widgets.Output()
with withdraw_out:
#    if CoinA_sale_contract.caller.finalized():
    display(post_sale_box)
#        #post_sale_box.layout.visibility = 'visible'
#    else:
#        #post_sale_box.layout.visibility = 'hidden'
#        pass

In [20]:
## On-click event widgets ##

# Update visuals
def on_button_clicked(b):
    with out:
        out.clear_output()
        port_df = generate_portfolio(risk_selector.value, time.value, time_selector.value)
        display(port_df['Portfolio Adj. Close'].hvplot())

button.on_click(on_button_clicked)


# Display dollar conversion and Confirm button
def coin_button_clicked(b):
    with confirm_out:
        if coin_text.value > cons_coin_supply.value:
            coin_text.value = cons_coin_supply.value
        else:
            pass
        
        dollar_conversion.value = "{:,}".format((coin_text.value / (10 ** coinA_contract.caller.decimals())) * eth_rate)
        confirm_out.clear_output()
        
        confirm_box.layout.visibility = 'visible'
        display(confirm_box)
    
coin_button.on_click(coin_button_clicked)


# Declare event when Confirm button is clicked - Activate Buy Tokens function
def confirm_button_clicked(b):
    
    with coin_out:
        coin_out.clear_output()
        
        buyer_address = account_selector.value
            
        if CoinA_sale_contract.caller.capReached() or CoinA_sale_contract.caller.hasClosed():
            coin_text.disabled = True
            coin_button.disabled = True
                
        elif coin_text.value >= CoinA_sale_contract.caller.cap() - CoinA_sale_contract.caller.weiRaised():
                
            nonce = w3.eth.getTransactionCount(buyer_address)
            remaining_supply = CoinA_sale_contract.caller.cap() - CoinA_sale_contract.caller.weiRaised()
            
            transaction = CoinA_sale_contract.functions.buyTokens(
                buyer_address).buildTransaction(
                {
                    'value': remaining_supply,
                    'gas': 3000000,
                    'nonce': nonce
                }
            )
            signed_txn = w3.eth.account.signTransaction(transaction, key_input.value)
            txn_hash = w3.eth.sendRawTransaction(signed_txn.rawTransaction)
            tx_receipt = w3.eth.waitForTransactionReceipt(txn_hash)
            
            cons_coin.value = CoinA_sale_contract.caller.balanceOf(buyer_address)
                
            cons_coin_supply.value = CoinA_sale_contract.caller.cap() - CoinA_sale_contract.caller.weiRaised()
                
            coin_text.disabled = True
            coin_button.disabled = True
                
        else:
            nonce = w3.eth.getTransactionCount(buyer_address)
            
            transaction = CoinA_sale_contract.functions.buyTokens(
                buyer_address).buildTransaction(
                {
                    'value': coin_text.value,
                    'gas': 3000000,
                    'nonce': nonce
                }
            )
            signed_txn = w3.eth.account.signTransaction(transaction, key_input.value)
            txn_hash = w3.eth.sendRawTransaction(signed_txn.rawTransaction)
            tx_receipt = w3.eth.waitForTransactionReceipt(txn_hash)
                
            cons_coin.value = CoinA_sale_contract.caller.balanceOf(buyer_address)
                
            cons_coin_supply.value = CoinA_sale_contract.caller.cap() - CoinA_sale_contract.caller.weiRaised()
        
        coin_text.value = 0
        confirm_box.layout.visibility = 'hidden'
            
        display(coins_box)
            
confirm_button.on_click(confirm_button_clicked)


# Declare event when withdraw button is clicked - withdraw tokens that were previously purchased
def withdraw_button_clicked(b):
    with withdraw_out:
        withdraw_address = withdraw_account.value
        withdraw_key = withdraw_key_input.value
        withdraw_out.clear_output()
        #post_sale_box.layout.visibility = 'visible'
        display(post_sale_box)
        
        withdraw = CoinA_sale_contract.functions.withdrawTokens(withdraw_address).buildTransaction(
            {
                'gas': 3000000,
                'nonce': w3.eth.getTransactionCount(withdraw_address)
            }
        )
        signed_txn_withdraw = w3.eth.account.signTransaction(withdraw, withdraw_key_input.value)
        txn_hash_withdraw = w3.eth.sendRawTransaction(signed_txn_withdraw.rawTransaction)
        tx_receipt_withdraw = w3.eth.waitForTransactionReceipt(txn_hash_withdraw)
    
withdraw_button.on_click(withdraw_button_clicked)


# Retrieve token balance
def token_balance_button_clicked(b):
    with withdraw_out:
        balance_address = withdraw_account.value
        token_balance.value = coinA_contract.caller.balanceOf(balance_address)
        
token_balance_button.on_click(token_balance_button_clicked)

In [21]:
# Organizes the widgets into layout boxes
interactions = widgets.HBox([risk_selector, button])
time_interactions = widgets.HBox([time, time_selector])
user_input = widgets.VBox([interactions, time_interactions])
user_box = widgets.VBox([user_input, out])

# Complete dashboard
dash = widgets.VBox([user_box, coins_box, withdraw_out], layout={'border': '2px solid black'})

In [22]:
# Sets a timer to automatically finalize the crowdsale when closing time has been reached.
now = datetime.now()
closingTime = CoinA_sale_contract.caller.closingTime()
time_window = (datetime.fromtimestamp(closingTime) - now)
seconds = time_window.seconds + 1

# Activates Finalize function automatically when closintTime is reached.
# Then displays post-sale widgets.
def finalize_sale():
    time_till_finalize.cancel()
    
    #if CoinA_sale_contract.caller.goalReached():
        
    try:
        transaction_fin = CoinA_sale_contract.functions.finalize().buildTransaction(
            {
                'gas': 3000000,
                'nonce': w3.eth.getTransactionCount(owner_address)
            }
        )
        signed_txn_fin = w3.eth.account.signTransaction(
            transaction_fin, owner_private_key
        )
        txn_hash_fin = w3.eth.sendRawTransaction(signed_txn_fin.rawTransaction)
        tx_receipt_fin = w3.eth.waitForTransactionReceipt(txn_hash_fin)

    except:
        CoinA_sale_contract.caller.finalized()
            
    #else: # Something should happen if funds are not raised when time ends
        
        #print('Funds not raised')
    
    coin_text.disabled = True
    coin_button.disabled = True
    account_selector.disabled = True
    account_selector.value = ''
    key_input.disabled = True
    key_input.value = ''
    
    coins_box.layout.visibility = 'hidden'
    post_sale_box.layout.visibility = 'visible'
    
    #with withdraw_out:
    #    display(post_sale_box)
    

time_till_finalize = Timer(seconds, finalize_sale)

time_till_finalize.start()

In [23]:
dash

In [25]:
CoinA_sale_contract.caller.finalized()

False